In [106]:
import warnings
warnings.filterwarnings('ignore')
from category_encoders import LeaveOneOutEncoder,WOEEncoder
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme(palette = "Dark2")
my_colors = [(0.10588235294117647, 0.6196078431372549, 0.4666666666666667),
  (0.8509803921568627, 0.37254901960784315, 0.00784313725490196)]
pd.set_option('display.max_columns', None)
from itertools import chain, combinations
import math


In [107]:
import importlib
import Model_functions
importlib.reload(Model_functions)
from Model_functions import Model_class 

In [108]:
# from xgboost import XGBClassifier
# from sklearn.metrics import roc_auc_score
# from sklearn.metrics import mean_absolute_error
# from catboost import CatBoostClassifier, Pool
# from keras.models import Sequential
# from keras.layers import Dense, Activation, Dropout
# import tensorflow as tensorflow
# from lightgbm import LGBMClassifier
# from sklearn.preprocessing import StandardScaler

### **5 Additional Features**
We will add some additional features to our model in order to improve its performance.

In [109]:
df = pd.read_csv("df_processed.csv")
df

,order_item_id,order_date,delivery_date,item_id,size,item_color,brand_id,item_price,user_id,user_title,user_dob,user_state,user_reg_date,return,delivery_time,order_id,user_age,user_reg_age,order_weekday,delivery_weekday,order_month,delivery_month,order_day,delivery_day,order_week,delivery_week,order_item_count,order_sum,average_item_price_order,order_number_same_item_id,order_number_different_item_id,order_number_same_size,order_number_different_size,order_number_same_item_color,order_number_different_item_color,order_number_same_brand_id,order_number_different_brand_id,order_number_same_item_id_size,order_number_different_item_id_size,order_number_same_item_id_item_color,order_number_different_item_id_item_color,order_number_same_size_item_color,order_number_different_size_item_color,order_number_same_size_brand_id,order_number_different_size_brand_id,order_number_same_item_color_brand_id,order_number_different_item_color_brand_id,order_number_same_item_id_size_item_color,order_number_different_item_id_size_item_color,order_number_same_size_item_color_brand_id,order_number_different_size_item_color_brand_id
0,1,2012-04-01,2012-04-03,186,s,denim,25,69.90,794,Mrs,1965-01-06,Bad-Wue,2011-04-25,0.0,2,2012-04-01_794,47,342,6,1,4,4,1,3,13,14,3,209.8,69.93,1,2,1,2,1,2,1,2,1,2,1,2,1,2,1,2,1,2,1,2,1,2
1,2,2012-04-01,2012-04-03,71,unsized,ocher,21,69.95,794,Mrs,1965-01-06,Bad-Wue,2011-04-25,1.0,2,2012-04-01_794,47,342,6,1,4,4,1,3,13,14,3,209.8,69.93,2,1,2,1,1,2,2,1,2,1,1,2,1,2,2,1,1,2,1,2,1,2
2,3,2012-04-01,2012-04-03,71,unsized,curry,21,69.95,794,Mrs,1965-01-06,Bad-Wue,2011-04-25,1.0,2,2012-04-01_794,47,342,6,1,4,4,1,3,13,14,3,209.8,69.93,2,1,2,1,1,2,2,1,2,1,1,2,1,2,2,1,1,2,1,2,1,2
3,4,2012-04-02,2012-04-06,22,s,green,14,39.90,808,Mrs,1959-11-09,Saxony,2012-01-04,0.0,4,2012-04-02_808,52,89,0,4,4,4,2,6,14,14,1,39.9,39.90,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0
4,5,2012-04-02,2012-04-06,151,s,black,53,29.90,825,Mrs,1964-07-11,S-Holstein,2011-02-16,0.0,4,2012-04-02_825,48,411,0,4,4,4,2,6,14,14,3,249.7,83.23,1,2,2,1,2,1,1,2,1,2,1,2,2,1,1,2,1,2,1,2,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
531165,50074,2013-04-29,2013-05-03,2342,s,terracotta,5,69.90,91920,Mrs,1962-03-08,Bayern,2013-04-29,NaN,4,2013-04-29_91920,51,0,0,4,4,5,29,3,18,18,9,748.2,83.13,1,8,5,4,3,6,4,5,1,8,1,8,3,6,4,5,3,6,1,8,3,6
531166,50075,2013-04-29,2013-05-03,2505,s,terracotta,5,64.90,91920,Mrs,1962-03-08,Bayern,2013-04-29,NaN,4,2013-04-29_91920,51,0,0,4,4,5,29,3,18,18,9,748.2,83.13,2,7,5,4,3,6,4,5,2,7,2,7,3,6,4,5,3,6,2,7,3,6
531167,50076,2013-04-28,2013-05-02,2470,l,white,5,79.90,85095,Mrs,1950-02-14,Berlin,2013-03-24,NaN,4,2013-04-28_85095,63,35,6,3,4,5,28,2,17,18,1,79.9,79.90,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0
531168,50077,2013-04-28,2013-05-02,2452,m,white,5,59.90,91922,Mrs,1969-11-27,Bburg,2013-04-28,NaN,4,2013-04-28_91922,43,0,6,3,4,5,28,2,17,18,2,119.8,59.90,2,0,2,0,1,1,2,0,2,0,1,1,1,1,2,0,1,1,1,1,1,1


In [110]:
low_cardinality_cat_cols = ["size","item_color","user_title","user_state"]
high_cardinality_cat_cols = ["item_id","brand_id","user_id"]
cat_cols = low_cardinality_cat_cols + high_cardinality_cat_cols

In [111]:
# Get the real class predictions for the test set
# real_class = pd.read_csv("orders_realclass.txt", delimiter=";")
# real_class = real_class.astype({"orderItemID":int,"returnShipment":int})
# df_valid.reset_index(drop=True,inplace=True)
# df_valid["return"] = real_class["returnShipment"]

In [112]:
# Combine the training and former testing sets into one dataframe
# df_train_new = pd.concat([df_train,df_test],axis=0)
# # Encoding the categorical variables
# df_train_encoded, df_test_encoded, encoder = encode_categorical_columns(df_train_new,df_valid,cat_cols,0.1)

### **Additional Features**

In [113]:
# Get number of unique values in a column
list_include = ["item_id","size","order_id","brand_id","item_color"]
column_name= {"item_id":"order_item_id_nunique","size":"order_size_nunique","brand_id":"order_brand_id_nunique","item_color":"order_item_color_nunique"}
df_group = df[list_include].groupby(["order_id"]).agg({"item_id":"nunique","size":"nunique","brand_id":"nunique","item_color":"nunique"}).reset_index().rename(columns= column_name)
df = df.merge(df_group,on="order_id",how="left")

In [114]:
# Same item_id different colors, sizes
df_group = df[list_include].groupby(["order_id","item_id"]).agg({"item_color":"nunique","size":"nunique"}).reset_index().rename(columns= {"item_color":"order_item_id_color_nunique","size":"order_item_id_size_nunique"})
df = df.merge(df_group,on=["order_id","item_id"],how="left")

In [115]:
# Same brand_id different colors, sizes, item_id
df_group = df[list_include].groupby(["order_id","brand_id"]).agg({"item_color":"nunique","size":"nunique","item_id":"nunique"}).reset_index().rename(columns= {"item_color":"order_brand_id_color_nunique","size":"order_brand_id_size_nunique","item_id":"order_brand_id_item_id_nunique"})
df = df.merge(df_group,on=["order_id","brand_id"],how="left")

In [116]:
stats_to_agg = ["mean","std","min","max","sum","count","median","mad"]
# Item id price statistics
df_group = df[["item_id","item_price"]].groupby(["item_id"]).agg(stats_to_agg).reset_index().round(2)
df_group.columns = ['_item_id_'.join(col) for col in df_group.columns]
df_group.rename(columns= {"item_id_item_id_":"item_id"},inplace=True)
df_group.fillna(0,inplace=True)
df = df.merge(df_group,on="item_id",how="left")

In [117]:
# user_id price statistics
df_group = df[["user_id","item_price"]].groupby(["user_id"]).agg(stats_to_agg).reset_index().round(2)
# df_group.columns = ['_'.join(col) for col in df_group.columns]
df_group.columns = ['_user_id_'.join(col) for col in df_group.columns]
df_group.rename(columns= {"user_id_user_id_":"user_id"},inplace=True)
df_group.fillna(0,inplace=True)
df = df.merge(df_group,on="user_id",how="left")

In [118]:
# brand_id price statistics
df_group = df[["brand_id","item_price"]].groupby(["brand_id"]).agg(stats_to_agg).reset_index().round(2)
df_group.columns = ['_brand_id_'.join(col) for col in df_group.columns]
df_group.rename(columns= {"brand_id_brand_id_":"brand_id"},inplace=True)
df_group.fillna(0,inplace=True)
df = df.merge(df_group,on="brand_id",how="left")

In [119]:
# Get the difference between the min, max and mean of the item_price column and the item_price column
min_max_mean = ["min","max","mean"]
for i in df.columns:
    for j in min_max_mean:
        if j in i:
            df["price-"+ i] = df[i] - df["item_price"] 

In [120]:
# Add new categorical variables to cat_cols
cat_cols += ["mode_item_id","mode_size","mode_brand_id","mode_item_color",]

In [121]:
# Get the most frequent values for user_id
df_group = df[["item_id","size","brand_id","item_color","user_id"]].groupby(["user_id"]).agg(lambda x: pd.Series.mode(x)[0])
df_group = df_group.add_prefix("mode_")
df = df.merge(df_group,on="user_id",how="left")

In [123]:
df

,order_item_id,order_date,delivery_date,item_id,size,item_color,brand_id,item_price,user_id,user_title,user_dob,user_state,user_reg_date,return,delivery_time,order_id,user_age,user_reg_age,order_weekday,delivery_weekday,order_month,delivery_month,order_day,delivery_day,order_week,delivery_week,order_item_count,order_sum,average_item_price_order,order_number_same_item_id,order_number_different_item_id,order_number_same_size,order_number_different_size,order_number_same_item_color,order_number_different_item_color,order_number_same_brand_id,order_number_different_brand_id,order_number_same_item_id_size,order_number_different_item_id_size,order_number_same_item_id_item_color,order_number_different_item_id_item_color,order_number_same_size_item_color,order_number_different_size_item_color,order_number_same_size_brand_id,order_number_different_size_brand_id,order_number_same_item_color_brand_id,order_number_different_item_color_brand_id,order_number_same_item_id_size_item_color,order_number_different_item_id_size_item_color,order_number_same_size_item_color_brand_id,order_number_different_size_item_color_brand_id,order_item_id_nunique,order_size_nunique,order_brand_id_nunique,order_item_color_nunique,order_item_id_color_nunique,order_item_id_size_nunique,order_brand_id_color_nunique,order_brand_id_size_nunique,order_brand_id_item_id_nunique,item_price_item_id_mean,item_price_item_id_std,item_price_item_id_min,item_price_item_id_max,item_price_item_id_sum,item_price_item_id_count,item_price_item_id_median,item_price_item_id_mad,item_price_user_id_mean,item_price_user_id_std,item_price_user_id_min,item_price_user_id_max,item_price_user_id_sum,item_price_user_id_count,item_price_user_id_median,item_price_user_id_mad,item_price_brand_id_mean,item_price_brand_id_std,item_price_brand_id_min,item_price_brand_id_max,item_price_brand_id_sum,item_price_brand_id_count,item_price_brand_id_median,item_price_brand_id_mad,price-item_price_item_id_mean,price-item_price_item_id_min,price-item_price_item_id_max,price-item_price_user_id_mean,price-item_price_user_id_min,price-item_price_user_id_max,price-item_price_brand_id_mean,price-item_price_brand_id_min,price-item_price_brand_id_max,mode_item_id,mode_size,mode_brand_id,mode_item_color
0,1,2012-04-01,2012-04-03,186,s,denim,25,69.90,794,Mrs,1965-01-06,Bad-Wue,2011-04-25,0.0,2,2012-04-01_794,47,342,6,1,4,4,1,3,13,14,3,209.8,69.93,1,2,1,2,1,2,1,2,1,2,1,2,1,2,1,2,1,2,1,2,1,2,2,2,2,3,1,1,1,1,1,69.47,3.41,39.90,69.90,21606.68,311,69.90,0.83,62.65,24.51,0.0,89.9,689.1,11,69.9,16.52,55.23,13.65,24.9,79.90,282536.19,5116,59.90,11.66,-0.43,-30.0,0.0,-7.25,-69.90,20.00,-14.67,-45.00,10.0,71,s,20,white
1,2,2012-04-01,2012-04-03,71,unsized,ocher,21,69.95,794,Mrs,1965-01-06,Bad-Wue,2011-04-25,1.0,2,2012-04-01_794,47,342,6,1,4,4,1,3,13,14,3,209.8,69.93,2,1,2,1,1,2,2,1,2,1,1,2,1,2,2,1,1,2,1,2,1,2,2,2,2,3,2,1,2,1,1,57.95,13.28,34.95,69.95,59108.70,1020,69.95,12.35,62.65,24.51,0.0,89.9,689.1,11,69.9,16.52,74.54,30.32,24.9,179.95,478991.63,6426,69.95,22.66,-12.00,-35.0,0.0,-7.30,-69.95,19.95,4.59,-45.05,110.0,71,s,20,white
2,3,2012-04-01,2012-04-03,71,unsized,curry,21,69.95,794,Mrs,1965-01-06,Bad-Wue,2011-04-25,1.0,2,2012-04-01_794,47,342,6,1,4,4,1,3,13,14,3,209.8,69.93,2,1,2,1,1,2,2,1,2,1,1,2,1,2,2,1,1,2,1,2,1,2,2,2,2,3,2,1,2,1,1,57.95,13.28,34.95,69.95,59108.70,1020,69.95,12.35,62.65,24.51,0.0,89.9,689.1,11,69.9,16.52,74.54,30.32,24.9,179.95,478991.63,6426,69.95,22.66,-12.00,-35.0,0.0,-7.30,-69.95,19.95,4.59,-45.05,110.0,71,s,20,white
3,4,2012-04-02,2012-04-06,22,s,green,14,39.90,808,Mrs,1959-11-09,Saxony,2012-01-04,0.0,4,2012-04-02_808,52,89,0,4,4,4,2,6,14,14,1,39.9,39.90,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,1,1,1,1,1,1,1,1,31.10,8.68,19.90,39.90,68669.20,2208,29.90,8.41,39.90,0.00,39.9,39.9,39.9,1,39.9,0.00,55.16,19.34,0.0,89.90,629751.00,11417,49.90,16.21,-8.80,-20.0,0.0,0.00,0.00,0.00,15.26,-39.90,50.0,22,s,14,green
4,5,2012-04-02,2012-04-06,151,s,black,53,29.90,825,Mrs,1964-07-11,S-Holstein,2011-02-

In [124]:
# Instantiate the model class
Model = Model_class(df)

In [125]:
# Split the data into train and test sets
df_train,df_test, df_valid = Model.split_data(False)
# Encode the categorical variables using leave-one-out encoding
df_train_encoded,df_test_encoded,encoder = Model.LOE_Encoder(df_train,df_test, cat_cols,0.1,True)

We are going to use both the CPU and GPU versions of the **catboost** model, to see which one is better. The GPU version of the model is faster and can thus be used with a higher amount of iterations without taking too much time. Both versions have different results depending on the hardware we use for the model since the GPU and CPU versions use different sets and values of hyperparameters.

In [126]:
catboost_params_cpu = {'iterations': 150, 'learning_rate': 0.1, 'depth': 10, 'loss_function': 'Logloss', 'random_seed': 42}
catboost_model_cpu, catboost_Y_pred_cpu, catboost_mae_cpu, catboost_Y_pred_proba_cpu = Model.catboost(df_train,df_test,catboost_params_cpu,0,cat_cols)
print("Catboost_cpu MAE on test set:", catboost_mae_cpu)

Catboost_cpu MAE on test set: 0.3256


In [132]:
catboost_params_gpu = {'iterations': 250, 'learning_rate': 0.10, 'depth': 10, 'loss_function': 'Logloss', 'random_seed': 42,"task_type":"GPU"}
catboost_model_gpu, catboost_Y_pred_gpu, catboost_mae_gpu, catboost_Y_pred_proba_gpu = Model.catboost(df_train,df_test,catboost_params_gpu,1,cat_cols)
print("Catboost_gpu MAE on test set:", catboost_mae_gpu)

Catboost_gpu MAE on test set: 0.3233


In [133]:
cat_boost_importance_cpu = Model.get_feature_importance(catboost_model_cpu,df_train)
cat_boost_importance_cpu[:10]

[('delivery_time', 15.655995287477024),
 ('user_id', 13.729875442954917),
 ('delivery_weekday', 8.11639207135039),
 ('item_id', 7.98920008977104),
 ('order_sum', 4.30900885146087),
 ('order_item_id_size_nunique', 2.0619282259540457),
 ('item_price_user_id_sum', 1.7454182862984993),
 ('brand_id', 1.6629908750030005),
 ('user_reg_age', 1.4009884844594236),
 ('price-item_price_item_id_mean', 1.3656404019562416)]

In [134]:
cat_boost_importance_gpu = Model.get_feature_importance(catboost_model_gpu,df_train)
cat_boost_importance_gpu[:10]

[('user_id', 14.531034469123357),
 ('delivery_time', 13.689771915377879),
 ('item_id', 8.07958561191205),
 ('delivery_weekday', 5.845689595697923),
 ('order_sum', 3.5869563574804153),
 ('item_color', 2.694837238155702),
 ('size', 2.2788300429387798),
 ('user_reg_age', 1.9225835898390349),
 ('user_age', 1.714246081298896),
 ('price-item_price_item_id_mean', 1.68478855761503)]

Judging from both feature importance results, we can conclude that for the CatBoost model the numeric variable **delivery_time** has one of the highest importance. We can hypothesize that engineering this variable will help the model to have a higher predictive power.

In [135]:
xgboost_params = {'max_depth': 9, 'learning_rate': 0.15, 'n_estimators': 50, 'objective': 'binary:logistic', 'random_state': 42,"objective":"binary:logistic"}
xgb_model, xgb_Y_pred, xgb_mae, xgb_Y_pred_proba = Model.xgboost(df_train_encoded,df_test_encoded,xgboost_params,0)
print("Xgboost MAE on test set:", xgb_mae)

Xgboost MAE on test set: 0.3305


In [136]:
xgboost_boost_importance = Model.get_feature_importance(xgb_model,df_train_encoded)
xgboost_boost_importance[:10]

[('user_id_LOE_encoded', 0.12860776),
 ('order_item_count', 0.08825954),
 ('item_id_LOE_encoded', 0.07009572),
 ('delivery_weekday', 0.065749936),
 ('order_item_id_size_nunique', 0.023972025),
 ('delivery_time', 0.02366515),
 ('order_number_same_item_color_brand_id', 0.020891557),
 ('order_sum', 0.019531693),
 ('order_number_same_item_id_item_color', 0.018995743),
 ('order_number_different_item_id_size_item_color', 0.018550958)]

In [137]:
lightgbm_params = { "learning_rate":0.15, "n_estimators":150, "objective":"binary","random_state":42, "colsample_bytree" : 0.8}
lightgbm_model, lightgbm_Y_pred, lightgbm_mae, lightgbm_Y_pred_prob = Model.lightgmb(df_train, df_test, lightgbm_params, 0,cat_cols)
print("Lightgbm MAE on test set:", lightgbm_mae)

Lightgbm MAE on test set: 0.3319


In [138]:
Model.get_feature_importance(lightgbm_model,df_train)[:10]

[('user_id', 2167),
 ('item_id', 1045),
 ('mode_item_id', 774),
 ('delivery_time', 82),
 ('brand_id', 58),
 ('delivery_weekday', 41),
 ('order_sum', 39),
 ('mode_brand_id', 30),
 ('item_price_user_id_sum', 24),
 ('item_color', 23)]

In [145]:
mae_dict = {"Catboost_native_CPU":catboost_mae_cpu,"Catboost_native_GPU":catboost_mae_gpu,"Xgboost":xgb_mae,"Lightgbm_native":lightgbm_mae}
model_results = pd.DataFrame(mae_dict,index=["MAE_2"]).round(4)
model_results.sort_values(by="MAE_2",ascending=True,inplace=True,axis=1)
model_results = model_results.transpose()
model_results.index.set_names(["Model"],inplace=True)
model_results.reset_index(inplace=True)
model_results

,Model,MAE_2
0,Catboost_native_GPU,0.3233
1,Catboost_native_CPU,0.3256
2,Xgboost,0.3305
3,Lightgbm_native,0.3319


In [146]:
# Read previous model performance
model_performance = pd.read_csv("Model_Performances/Model_performance.csv")
model_results = pd.merge(model_results,model_performance,on="Model",how="left")
model_results["MAE_Improvement"] = model_results["MAE_1"] - model_results["MAE_2"]
model_results

,Model,MAE_2,MAE_1,MAE_Improvement
0,Catboost_native_GPU,0.3233,0.3276,0.0043
1,Catboost_native_CPU,0.3256,0.3276,0.0020
2,Xgboost,0.3305,0.3351,0.0046
3,Lightgbm_native,0.3319,0.3305,-0.0014


Overall the additional features did improve the performance of all models except for the **LightGBM** model. The Catboost model with its **GPU** version is the best performing model followed closely by its **CPU** version. The **Xgboost** comes in third, both **Catboost_GPU** and **Xgboost** got their performance improved by around **0.5%** due to the additional features. To test our hypothesis that generating additional features using the variable **delivery_time** will help the models to have a higher predictive power, we will aggregate different statistics for the **delivery_time** variable and see if the results improve.

In [147]:
cols_to_agg = ["item_id","size","brand_id","item_color","user_id"]
stats_to_agg = ["mean","std","min","max","median","mad"]
# Loop through the columns to aggregate
for i in cols_to_agg:
    df_group = df[[i,"delivery_time"]].groupby([i]).agg(stats_to_agg).reset_index().round(2)
    df_group.columns = [str("_" + i +"_").join(col) for col in df_group.columns]
    df_group.rename(columns= {i+"_"+i+"_":i},inplace=True)
    df_group.fillna(0,inplace=True)
    df = df.merge(df_group,on=i,how="left")    

In [148]:
df

,order_item_id,order_date,delivery_date,item_id,size,item_color,brand_id,item_price,user_id,user_title,user_dob,user_state,user_reg_date,return,delivery_time,order_id,user_age,user_reg_age,order_weekday,delivery_weekday,order_month,delivery_month,order_day,delivery_day,order_week,delivery_week,order_item_count,order_sum,average_item_price_order,order_number_same_item_id,order_number_different_item_id,order_number_same_size,order_number_different_size,order_number_same_item_color,order_number_different_item_color,order_number_same_brand_id,order_number_different_brand_id,order_number_same_item_id_size,order_number_different_item_id_size,order_number_same_item_id_item_color,order_number_different_item_id_item_color,order_number_same_size_item_color,order_number_different_size_item_color,order_number_same_size_brand_id,order_number_different_size_brand_id,order_number_same_item_color_brand_id,order_number_different_item_color_brand_id,order_number_same_item_id_size_item_color,order_number_different_item_id_size_item_color,order_number_same_size_item_color_brand_id,order_number_different_size_item_color_brand_id,order_item_id_nunique,order_size_nunique,order_brand_id_nunique,order_item_color_nunique,order_item_id_color_nunique,order_item_id_size_nunique,order_brand_id_color_nunique,order_brand_id_size_nunique,order_brand_id_item_id_nunique,item_price_item_id_mean,item_price_item_id_std,item_price_item_id_min,item_price_item_id_max,item_price_item_id_sum,item_price_item_id_count,item_price_item_id_median,item_price_item_id_mad,item_price_user_id_mean,item_price_user_id_std,item_price_user_id_min,item_price_user_id_max,item_price_user_id_sum,item_price_user_id_count,item_price_user_id_median,item_price_user_id_mad,item_price_brand_id_mean,item_price_brand_id_std,item_price_brand_id_min,item_price_brand_id_max,item_price_brand_id_sum,item_price_brand_id_count,item_price_brand_id_median,item_price_brand_id_mad,price-item_price_item_id_mean,price-item_price_item_id_min,price-item_price_item_id_max,price-item_price_user_id_mean,price-item_price_user_id_min,price-item_price_user_id_max,price-item_price_brand_id_mean,price-item_price_brand_id_min,price-item_price_brand_id_max,mode_item_id,mode_size,mode_brand_id,mode_item_color,delivery_time_item_id_mean,delivery_time_item_id_std,delivery_time_item_id_min,delivery_time_item_id_max,delivery_time_item_id_median,delivery_time_item_id_mad,delivery_time_size_mean,delivery_time_size_std,delivery_time_size_min,delivery_time_size_max,delivery_time_size_median,delivery_time_size_mad,delivery_time_brand_id_mean,delivery_time_brand_id_std,delivery_time_brand_id_min,delivery_time_brand_id_max,delivery_time_brand_id_median,delivery_time_brand_id_mad,delivery_time_item_color_mean,delivery_time_item_color_std,delivery_time_item_color_min,delivery_time_item_color_max,delivery_time_item_color_median,delivery_time_item_color_mad,delivery_time_user_id_mean,delivery_time_user_id_std,delivery_time_user_id_min,delivery_time_user_id_max,delivery_time_user_id_median,delivery_time_user_id_mad
0,1,2012-04-01,2012-04-03,186,s,denim,25,69.90,794,Mrs,1965-01-06,Bad-Wue,2011-04-25,0.0,2,2012-04-01_794,47,342,6,1,4,4,1,3,13,14,3,209.8,69.93,1,2,1,2,1,2,1,2,1,2,1,2,1,2,1,2,1,2,1,2,1,2,2,2,2,3,1,1,1,1,1,69.47,3.41,39.90,69.90,21606.68,311,69.90,0.83,62.65,24.51,0.0,89.9,689.1,11,69.9,16.52,55.23,13.65,24.9,79.90,282536.19,5116,59.90,11.66,-0.43,-30.0,0.0,-7.25,-69.90,20.00,-14.67,-45.00,10.0,71,s,20,white,5.04,5.80,1,30,3.0,3.61,10.38,16.45,0,71,4.0,11.27,4.21,5.15,0,71,3.0,2.65,9.80,15.10,0,71,4.0,10.38,3.64,3.38,1,13,3.0,2.20
1,2,2012-04-01,2012-04-03,71,unsized,ocher,21,69.95,794,Mrs,1965-01-06,Bad-Wue,2011-04-25,1.0,2,2012-04-01_794,47,342,6,1,4,4,1,3,13,14,3,209.8,69.93,2,1,2,1,1,2,2,1,2,1,1,2,1,2,2,1,1,2,1,2,1,2,2,2,2,3,2,1,2,1,1,57.95,13.28,34.95,69.95,59108.70,1020,69.95,12.35,62.65,24.51,0.0,89.9,689.1,11,69.9,16.52,74.54,30.32,24.9,179.95,478991.63,6426,69.95,22.66,-12.00,-35.0,0.0,-7.30,-69.95,19.95,4.59,-45.0

In [149]:
# Instantiate the model class
Model = Model_class(df)

In [150]:
# Split the data into train and test sets
df_train,df_test, df_valid = Model.split_data(False)
# Encode the categorical variables using leave-one-out encoding
df_train_encoded,df_test_encoded,encoder = Model.LOE_Encoder(df_train,df_test, cat_cols,0.1,True)

In [152]:
catboost_params_cpu = {'iterations': 150, 'learning_rate': 0.1, 'depth': 10, 'loss_function': 'Logloss', 'random_seed': 42}
catboost_model_cpu, catboost_Y_pred_cpu, catboost_mae_cpu, catboost_Y_pred_proba_cpu = Model.catboost(df_train,df_test,catboost_params_cpu,1,cat_cols)
print("Catboost_CPU MAE on test set:", catboost_mae_cpu)

Catboost_CPU MAE on test set: 0.3196


In [169]:
catboost_params_gpu = {'iterations': 225, 'learning_rate': 0.1, 'depth': 10, 'loss_function': 'Logloss', 'random_seed': 42,"task_type":"GPU"}
catboost_model_gpu, catboost_Y_pred_gpu, catboost_mae_gpu, catboost_Y_pred_proba_gpu = Model.catboost(df_train,df_test,catboost_params_gpu,1,cat_cols)
print("Catboost_GPU MAE on test set:", catboost_mae_gpu)

Catboost_GPU MAE on test set: 0.3191


In [154]:
Model.get_feature_importance(catboost_model_gpu,df_train)[:10]

[('user_id', 13.924722367936104),
 ('delivery_time', 13.833219987629844),
 ('item_id', 7.6197237308465615),
 ('delivery_weekday', 5.505180148476735),
 ('order_sum', 3.0715550492240578),
 ('size', 2.1400651458330824),
 ('item_color', 1.9389941723073219),
 ('user_reg_age', 1.6445762269416841),
 ('item_price_user_id_sum', 1.5792425544482258),
 ('delivery_time_user_id_min', 1.5672746568260756)]

In [155]:
Model.get_feature_importance(catboost_model_cpu,df_train)[:10]

[('delivery_time', 17.19151063817235),
 ('user_id', 13.058865545441632),
 ('delivery_weekday', 8.727475431484091),
 ('item_id', 7.16512763833561),
 ('order_sum', 3.496313758614527),
 ('order_item_id_size_nunique', 1.940389704700041),
 ('delivery_time_user_id_median', 1.7708780275511604),
 ('delivery_time_user_id_min', 1.668951272982231),
 ('brand_id', 1.6385683872863828),
 ('item_price_user_id_sum', 1.3648377543780637)]

We can see that some of our newly derived features have made it to the top of the feature importance results. Both **delivery_time_user_id_median** and **delivery_time_user_id_mean** seems to have a significant impact on the model.

In [156]:
xgboost_params = {'max_depth': 9, 'learning_rate': 0.15, 'n_estimators': 50, 'objective': 'binary:logistic', 'random_state': 42,"objective":"binary:logistic"}
xgb_model, xgb_Y_pred, xgb_mae, xgb_Y_pred_proba = Model.xgboost(df_train_encoded,df_test_encoded,xgboost_params,0)
print("Xgboost MAE on test set:", xgb_mae)

Xgboost MAE on test set: 0.3258


In [157]:
Model.get_feature_importance(xgb_model,df_train_encoded)[:10]

[('user_id_LOE_encoded', 0.10987093),
 ('order_item_count', 0.06970507),
 ('item_id_LOE_encoded', 0.061749194),
 ('delivery_weekday', 0.052036434),
 ('delivery_time', 0.022756113),
 ('order_item_id_size_nunique', 0.02052849),
 ('order_number_same_item_id_item_color', 0.017259905),
 ('item_price', 0.016240247),
 ('order_sum', 0.016216852),
 ('delivery_time_item_id_median', 0.013297142)]

In [158]:
lightgbm_params = { "learning_rate":0.15, "n_estimators":150, "objective":"binary","random_state":42, "colsample_bytree" : 0.8}
lightgbm_model, lightgbm_Y_pred, lightgbm_mae, lightgbm_Y_pred_prob = Model.lightgmb(df_train, df_test, lightgbm_params, 0,cat_cols)
print("Lightgbm MAE on test set:", lightgbm_mae)

Lightgbm MAE on test set: 0.3301


In [160]:
mae_dict_1 = {"Catboost_native_CPU":catboost_mae_cpu,"Catboost_native_GPU":catboost_mae_gpu,"Xgboost":xgb_mae,"Lightgbm_native":lightgbm_mae}
model_results_1 = pd.DataFrame(mae_dict_1,index=["MAE_3"]).round(4)
model_results_1.sort_values(by="MAE_3",ascending=True,inplace=True,axis=1)
model_results_1 = model_results_1.transpose()
model_results_1.index.set_names(["Model"],inplace=True)
model_results_1.reset_index(inplace=True)
model_results_1

,Model,MAE_3
0,Catboost_native_GPU,0.3191
1,Catboost_native_CPU,0.3196
2,Xgboost,0.3258
3,Lightgbm_native,0.3301


In [167]:
model_results_whole = pd.merge(model_results,model_results_1,on="Model",how="left")
model_results_whole["MAE_Improvement"] = model_results_whole["MAE_2"] - model_results_whole["MAE_3"]
model_results_whole.drop(columns=["MAE_1"],inplace=True)
model_results_whole = model_results_whole[["Model","MAE_2","MAE_3","MAE_Improvement"]]
model_results_whole

,Model,MAE_2,MAE_3,MAE_Improvement
0,Catboost_native_GPU,0.3233,0.3191,0.0042
1,Catboost_native_CPU,0.3256,0.3196,0.0060
2,Xgboost,0.3305,0.3258,0.0047
3,Lightgbm_native,0.3319,0.3301,0.0018


Overall the addition of the **delivery_time** features did improve the performance of all models and managed as well to reduce the difference between the **CPU** and **GPU** versions of Catboost.
LightGBM seems to not perform as well as the other models for that reason we will not use this model anymore.

In [170]:
model_results_whole[["Model","MAE_3"]].to_csv("Model_Performances/Model_performance_1.csv",index=False)

In [168]:
# df to csv
df.to_csv("df_127_columns.csv",index=False)